In [10]:
!nvidia-smi

Wed Apr  7 06:41:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1E.0 Off |                    0 |
| N/A   21C    P0    36W / 300W |     11MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [11]:
!pip install -q transformers
!pip3 install -q ipywidgets --user
!pip install torch torchvision torchaudio
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html
!pip install tqdm boto3 requests regex sentencepiece sacremoses

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
Looking in links: https://download.pytorch.org/whl/torch_stable.html
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [12]:
import time
import datetime
import os
import json
import re
import sys
import pprint
# Importing stock ml libraries
import numpy as np
import pandas as pd
from sklearn import metrics

import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch import cuda
import transformers
from transformers import DistilBertTokenizer, DistilBertModel, BertConfig



In [13]:
#MM - TODO: Uncomment for Manohar's run
# INPUT_PATH = '/kaggle/input/emo-ekman'
# INPUT_MODELS = '/kaggle/input/emo-semfr/models'
# MODEL_PATH = '/kaggle/working/models'
# OUTPUT_PATH = '/kaggle/working/results'
#DK - TODO: Uncomment for Devesh's run
INPUT_PATH = 'Data/'
INPUT_MODELS = './models'
MODEL_PATH = './models'
OUTPUT_PATH = './training_results/clean_runs'

if not os.path.exists(MODEL_PATH):
    os.mkdir(MODEL_PATH) 
if not os.path.exists(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH) 

#TODO - Change pretrained model
PRE_TRAINED_MODEL = 'distilbert-base-cased'
device = torch.device('cuda' if cuda.is_available() else 'cpu')


Motivation from https://github.com/DhavalTaunk08/NLP_scripts/blob/master/Transformers_multilabel_distilbert.ipynb  
https://mccormickml.com/2019/07/22/BERT-fine-tuning/#32-required-formatting  
https://towardsdatascience.com/how-to-use-bert-from-the-hugging-face-transformer-library-d373a22b0209  

https://colab.research.google.com/github/abhimishra91/transformers-tutorials/blob/master/transformers_multi_label_classification.ipynb#scrollTo=DegHNyIEQxB2


In [14]:
    
def format_time(seconds):
    return str(datetime.timedelta(seconds=seconds))

In [15]:
def load_file(file,sep='\t'):
    '''Load a given file, combine the labels into 1 field called labels
       and return a dataframe
       Args: file contains a sep delimited data. Expected format of the file is 
             Text followed by One Hot Encoded list of columns for each label
       Returns: dataframe containing 2 columns
                First column of Text 
                Second column - list of all labels 
    '''

    df = pd.read_csv(file,sep=sep)
    df['labels'] = df[df.columns[1:]].values.tolist()
    df = df[[df.columns[0],'labels']]
    return df

In [16]:
def load_corpusdata(path,corpus,samples=None,type='train', ext='tsv',sep='\t'):
    ''' Loads data for multiple corpus, samples from each corpus would be loaded 
        and returned as one dataframe.
        Files are expected to be in path + '/' + corpus + '_' + type + '.' + sep
        Args: path : starting path for all corpus files
              corpus : list of corpus to load data from
              samples : Either None or list of sizes to select from each corpus
                        If list specified, len should be same as length of corpus
                        If None - all records from corpus files are included
              type : Expected use is for training data from multiple corpus to be combined
              ext  : Extension
        Returns: Dataframe from all the corpus in samples specified      
    '''
    df_list = []
    if not samples:
        samples = [1000000] * len(corpus)
    for f,s in zip(corpus,samples):
        df = load_file(path +'/' + f + '_' + type + '.' + ext,sep)
        if s < df.shape[0]:
            df = df.sample(s)
        df_list.append(df)
    df_all = pd.concat(df_list).reset_index(drop=True)
    colnames = {k:k.lower() for k in df_all.columns}
    df_all.rename(columns=colnames,inplace=True)
    return df_all

In [17]:
class EmotionDataset(Dataset):
    '''Subclass f torch.util.data.Dataset abstract class to provide access to our data'''
    def __init__(self, dataframe, tokenizer, max_length=64):
        '''
            Args: dataframe - Dataframe containing 2 columns. First column should contain text
                              and second column should contain list of 1 hot-encoded labels
                  tokenizer - Tokenizer to use. Tokenizer must correspond to the BERT model used
        '''
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = self.data.iloc[:,0]
        self.targets = self.data.iloc[:,1]
        self.max_length = max_length
        self.labels_len = self.targets.str.len().max()

        

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ''' '''
        text = str(self.text[index])
        text = " ".join(text.split()) # Force convert different whitespaces into a single space

        # encode_plus referred to in most documents on web is now deprecated. 
        # Using __call__ instead
        inputs = self.tokenizer(
            text=text,
            text_pair=None,
            add_special_tokens=True,
            padding='max_length', 
            # While Max_Length is known for BERT depending on Model, 
            # total length impacts training time
            max_length=self.max_length,
            return_token_type_ids=True
        )
        # Inputs contains three keys
        ids = inputs['input_ids']
        mask = inputs['attention_mask']  
        token_ids = inputs['token_type_ids']

        return {
            'ids'            : torch.tensor(ids,dtype=torch.long),
            'mask'           : torch.tensor(mask,dtype=torch.long),
            'token_type_ids' : torch.tensor(token_ids,dtype=torch.long),
            'targets'        : torch.tensor(self.targets[index],dtype=torch.float)
        } 
        

In [18]:
class EmotionBERTModel(torch.nn.Module):
    def __init__(self,classes,model,drop_out=False):
        super(EmotionBERTModel,self).__init__()
        self.model = model 
        self.hidden_size = model.config.hidden_size
        self.classifier_pre = torch.nn.Linear(self.hidden_size,self.hidden_size)
        if drop_out:
            self.dropout = torch.nn.Dropout(drop_out)
        else:
            self.dropout = None
        self.classifier = torch.nn.Linear(self.hidden_size,classes)

    def forward(self,input_ids,attention_mask,token_type_ids):
        output_1 = self.model(input_ids=input_ids,attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.classifier_pre(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output


In [19]:
def get_tokenizer():
    return DistilBertTokenizer.from_pretrained(PRE_TRAINED_MODEL)

In [20]:
def get_basemodel(labels_len,drop_out):
    model = DistilBertModel.from_pretrained(PRE_TRAINED_MODEL)
    train_model = EmotionBERTModel(labels_len,model,drop_out=drop_out)
    return train_model

In [21]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [22]:
def train(loader,optimizer,model,epoch):
    '''Train the data for each epoch'''
    epoch_start = time.time()
    #Reset total loss
    total_train_loss = 0

    #Set the model to train mode. 
    # This only changes the mode, so dropout and batchnorm perform correctlt 
    model.train()
 #   device = get_device()
    for step,data in enumerate(loader):
        # Progress update every 40 batches.
        #Unpack the data
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device)
        

        # Zero out the previously calculated gradients before passing backward pass 
        model.zero_grad()
        # Perform a forward pass (evaluate the model on this training batch).
        outputs = model(ids, mask, token_type_ids)

#        optimizer.zero_grad()
        # Calculated the loss for this batch
        loss = loss_fn(outputs, targets)
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. 
        # `loss` is a Tensor containing a single value; 
        #  the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Report progress every 100 batches
        if step % 100 == 0 and not step == 0:
            print('Epoch {:>2}  Batch {:>5,}  of  {:>5,}.  Loss {:0.4f}  Elapsed: {:}.'.format(
                 epoch,
                 step, 
                 len(loader),
                 loss.item(),
                 format_time(time.time() - epoch_start)))        

#        optimizer.zero_grad()
        #Perform backward pass to calculate gradients
        loss.backward()
        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

    avg_train_loss = total_train_loss / len(loader) 
    elapsed_time = time.time() - epoch_start
    print ("Avg Training Loss {:0.4f}, Completed in {:} ".format(
        avg_train_loss,
        format_time(elapsed_time)
          ))
    return model,avg_train_loss,elapsed_time

In [23]:
def calc_output(loader,model):
    fin_targets=[]
    fin_outputs=[]
#    device = get_device()
    t0 = time.time()
    total_val_loss = 0
    # Put the model in evaluation mode. Dropout layers behave differently in evaluation mode
    model.eval()
    # Tell pytorch not to construct the compute graph during the forward pass, 
    # since this is only needed for backprop (training).
    with torch.no_grad():
        for step, data in enumerate(loader):
            #Unpack the data
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            # Perform forward pass and predict output
            outputs = model(ids, mask, token_type_ids)

            # Calculated the loss for this batch
            loss = loss_fn(outputs, targets)
            # Accumulate the validation loss over all of the batches so that we can
            # calculate the average loss at the end. 
            total_val_loss += loss.item()

            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
            if step % 40 == 0 and not step == 0:
                print('  Batch {:>5,}  of  {:>5,}.  Loss {:0.4f}  Elapsed: {:}.'.format(
                    step, 
                    len(loader),
                    loss.item(),
                    format_time(time.time() - t0)))        

    avg_val_loss = total_val_loss / len(loader)
    elapsed_time = time.time() - t0
    print ("Avg Validation Loss {:0.4f}, Completed in {:} ".format(
        avg_val_loss,
        format_time(elapsed_time)
          ))
    return fin_outputs, fin_targets, avg_val_loss,elapsed_time

In [24]:
#Calculating the score on the val set:

def generate_score(loader,model,batch_size=1):
    results = {"weighted":0,"micro":0,"macro":0,"samples":0}
    
    
    print("**** Loss ******")
    outputs, targets,val_loss,val_time = calc_output(loader,model)
    outputs = (np.array(outputs) >= 0.5).astype(float)
    print("**** Results ******")
    for k in results.keys():
        score = metrics.f1_score(targets, outputs,average = k) 
        print("F1 Score ({}) = {:10.6f}".format(k,score))
        results[k] = score
    results['outputs'] = outputs.tolist()
    results['targets'] = targets
    results['val_loss'] = val_loss
    results['val_time'] = val_time
    return results
            


In [25]:
def get_datareaders(train_data,batch_size=16,max_length=64):
    tokenizer = get_tokenizer()
    
    dataset = EmotionDataset(train_data,tokenizer,max_length=max_length)
    dl_params = {'batch_size': batch_size,'shuffle': True,'num_workers': 0 }
    loader = DataLoader(dataset,**dl_params)  
    return loader, dataset  

In [26]:
def train_model(train_data,val_data,trained_model:None, freeze_layers=None,
                epochs=4,batch_size=16,drop_out=0.3,lr=1e-05,max_length=64):

    train_stats = []
    train_loader, train_dataset = get_datareaders(train_data,batch_size,max_length=max_length)
    val_loader,val_dataset = get_datareaders(val_data,batch_size,max_length=max_length)

    if trained_model is None:
        trained_model = get_basemodel(train_dataset.labels_len,drop_out=drop_out)

    if freeze_layers:
        for name,param in trained_model.named_parameters():
            if any([fl in name for fl in freeze_layers]):
                param.requires_grad = False

    trained_model.to(device)

    optimizer = torch.optim.Adam(params = trained_model.parameters(), lr=lr)
    start_time = time.time()
    for epoch in range(epochs):
        print ('*****  Epoch {} *****'.format(epoch))
        print ('-----  Training -----')
        trained_model,train_loss,train_time = train(train_loader,optimizer,trained_model,epoch)
        print ('-----  Validation -----')
        results = generate_score(val_loader,trained_model)
        results['epoch'] = str(epoch)
        results['train_loss'] = train_loss
        results['train_time'] = train_time
        results['train_count'] = train_data.shape[0]
        results['val_count'] = val_data.shape[0]
        train_stats.append(results)
    
        
    return trained_model, train_stats

In [27]:
def test_model(test_data,model,epoch,max_length=64):
    test_loader,test_dataset = get_datareaders(test_data,batch_size=1,max_length=64)
    print ('-----  Testing -----')
    results = generate_score(test_loader,model)
    results['epoch'] = epoch
    return results

In [28]:
#TODO: Certainly Change TRIAL_RUN
# Setup models for experiments for the production run
# To ensure that model is setup correctly
# Set Trial Run to true. This will ensure all models to be run
#   Trial run would run on CPU typically in less than 10 minutes
# Then change TRIAL_RUN to False and run on GPU
metadata = {
    'TRIAL_RUN':False,
    'TRIAL_SIZE': 5,
    'TRIAL_EPOCHS': 1,
    'run_type':'ekman_final_positive',
    'description':'Run Ekman classification with Transfer learning from GoEmotions, SemEval and Friends datasets',
    'model_ext':'.pt',
    'dict_ext':'_inf.txt'
}
model_config = {
    'epochs'      :   4,
    'batch_size'  :  16,
    'max_length'  : 512,
    'lr'          : 4e-05,
    'drop_out'    : 0.2,
    'dataset_path':INPUT_PATH,
    'input_model_path': INPUT_MODELS,
    'model_path'  :MODEL_PATH,
    'output_path' :OUTPUT_PATH
}
experiments = {
    'GOEMO' :{
        'run'          :False,
        'description'  :'GoEmotions ',
        'train_corpus' : ['goemotions'],
        'test_corpus'  : ['tales'],
        'save_model'   :'emo_goemo',
        'save_stats'   :'stats_goemo'
    },
    'GOEMO_EMB_FR' :{
        'run'          :False,
        'description'  :'GoEmotions Freeze Embeddings',
        'load_model'   :None,
        'freeze_layers':['embeddings'],
        'train_corpus' : ['goemotions'],
        'train_samples': None,
        'test_corpus'  : ['tales'],
        'test_samples' : None,
        'save_model'   :'emo_goemo_emb_fr',
        'save_stats'   :'stats_goemo'
    },    
    'GOEMO_FR' :{
        'run'          :False,
        'description'  :'GoEmotions and Friends for training',
        'train_corpus' : ['goemotions','friends'],
        'test_corpus'  : ['tales'],
        'save_model'   :'emo_goemo_fr',
        'save_stats'   :'stats_goemo'
    },
    'GOEMO_SEM' :{
        'run'          :False,
        'description'  :'GoEmotions,SemEval for training',
        'train_corpus' : ['goemotions','semeval'],
        'test_corpus'  : ['tales'],
        'save_model'   :'emo_goemo_sem',
        'save_stats'   :'stats_goemo_sem'
    },
    'GOEMO_SEM_FR' :{
        'run'          :False,
        'description'  :'GoEmotions,SemEval and Friends for training',
        'train_corpus' : ['goemotions','semeval','friends'],
        'test_corpus'  : ['tales'],
        'save_model'   :'emo_goemo_sem_fr',
        'save_stats'   :'stats_goemo_sem_fr'
    },
    
    'TALES_100': {
        'run': True,
        'description': "TALES with ['tales'] 100 samples",
      'load_model': None,
      'train_corpus': ['tales'],
      'train_samples': [100],
      'test_corpus': ['tales'],
      'save_stats': 'stats_tales_100'},
    
    'TALES_200': {
        'run': True,
        'description': "TALES with ['tales'] 200 samples",
      'load_model': None,
      'train_corpus': ['tales'],
      'train_samples': [200],
      'test_corpus': ['tales'],
      'save_stats': 'stats_tales_200'},
    
    'TALES_300': {
        'run': True,
        'description': "TALES with ['tales'] 300 samples",
      'load_model': None,
      'train_corpus': ['tales'],
      'train_samples': [300],
      'test_corpus': ['tales'],
      'save_stats': 'stats_tales_300'},
    
     'TALES_400': {
        'run': True,
        'description': "TALES with ['tales'] 400 samples",
      'load_model': None,
      'train_corpus': ['tales'],
      'train_samples': [400],
      'test_corpus': ['tales'],
      'save_stats': 'stats_tales_400'},
            
    'TARGET_500': {
        'run': True,
        'description': "TARGET with ['tales'] 500 samples",
      'load_model': None,
      'train_corpus': ['tales'],
      'train_samples': [500],
      'test_corpus': ['tales'],
      'save_stats': 'stats_target_500'},
     
    'TALES_600': {
        'run': True,
        'description': "TALES with ['tales'] 600 samples",
      'load_model': None,
      'train_corpus': ['tales'],
      'train_samples': [600],
      'test_corpus': ['tales'],
      'save_stats': 'stats_tales_600'},
    
     'TALES_700': {
        'run': True,
        'description': "TALES with ['tales'] 700 samples",
      'load_model': None,
      'train_corpus': ['tales'],
      'train_samples': [700],
      'test_corpus': ['tales'],
      'save_stats': 'stats_tales_700'},
    
    'TALES_800': {
        'run': True,
        'description': "TALES with ['tales'] 800 samples",
      'load_model': None,
      'train_corpus': ['tales'],
      'train_samples': [800],
      'test_corpus': ['tales'],
      'save_stats': 'stats_tales_800'},
    
    'TALES_900': {
        'run': True,
        'description': "TALES with ['tales'] 900 samples",
      'load_model': None,
      'train_corpus': ['tales'],
      'train_samples': [900],
      'test_corpus': ['tales'],
      'save_stats': 'stats_tales_900'},
    
 'TALES_1000': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 1000 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [1000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_1000'},
    
 'TALES_1100': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 1100 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [1100],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_1100'},
 
    'TALES_1200': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 1200 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [1200],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_1200'},
    
    'TALES_1300': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 1300 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [1300],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_1300'},
    
    'TALES_1400': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 1400 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [1400],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_1400'},
    
    
 'TALES_1500': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 1500 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [1500],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_1500'},
    
    
    'TALES_1600': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 1600 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [1600],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_1600'},
    
      'TALES_1700': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 1700 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [1700],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_1700'},
    
    'TALES_1800': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 1800 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [1800],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_1800'},
    
    'TALES_1900': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 1900 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [1900],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_1900'},
    
    
 'TALES_2000': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 2000 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [2000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_2000'},
    
    'TALES_2100': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 2100 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [2100],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_2100'},
    
    
    'TALES_2200': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 2200 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [2200],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_2200'},
    
    'TALES_2300': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 2300 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [2300],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_2300'},
    
    'TALES_2400': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 2400 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [2400],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_2400'},
    
    'TALES_2500': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 2500 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [2500],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_2500'},
    
    'TALES_2600': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 2600 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [2600],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_2600'},
    
    'TALES_2700': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 2700 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [2700],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_2700'},
    
    'TALES_2800': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 2800 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [2800],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_2800'},
    
      'TALES_2900': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 2900 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [2900],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_2900'},
       
 'TALES_3000': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 3000 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [3000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_3000'},
    
    'TALES_3100': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 3100 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [3100],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_3100'},
    
    'TALES_3200': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 3200 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [3200],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_3200'},
    
    'TALES_3300': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 3300 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [3300],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_3300'},
    
     'TALES_3400': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 3400 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [3400],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_3400'},
    
       'TALES_3500': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 3400 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [3500],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_3500'},
    
     'TALES_3600': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 3600 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [3600],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_3600'},
    
     'TALES_3700': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 3700 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [3700],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_3700'},
    
     'TALES_3800': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 3800 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [3800],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_3800'},
    
       'TALES_3900': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 3900 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [3900],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_3900'},
    
    'TALES_4000': {'run': True,
  'name' : 'TALES',                 
  'description': "Target with ['tales'] 4000 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [4000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_4000'},
    
 'TALES_5000': {'run': True,
  'name' : 'TALES',                 
  'description': "Tales with ['tales'] 5000 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [5000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_5000'},
 'TALES_8000': {'run': True,
  'name' : 'TALES',                 
  'description': "Tales with ['tales'] 8000 samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': [8000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_8000'},
 'TALES_ALL': {'run': True,
  'name' : 'TALES',                 
  'description': "Tales with ['tales'] ALL samples",
  'load_model': None,
  'train_corpus': ['tales'],
  'train_samples': None,
  'test_corpus': ['tales'],
  'save_stats': 'stats_tales_all'},
    
    'GEMB_FR_3_5_100': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 100 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [100],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_100'},
    
      'GEMB_FR_3_5_200': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 200 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [200],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_200'},
    
    'GEMB_FR_3_5_300': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 300 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [300],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_300'},
    
    'GEMB_FR_3_5_400': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 400 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [400],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_400'},
    
    
 'GEMB_FR_3_5_500': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 500 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [500],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_500'},
 
    'GEMB_FR_3_5_600': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 600 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [600],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_600'},
    
    'GEMB_FR_3_5_700': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 700 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [700],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_700'},
    
      'GEMB_FR_3_5_800': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 800 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [800],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_800'},
    
     'GEMB_FR_3_5_900': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 900 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [900],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_900'},
 
   'GEMB_FR_3_5_1000': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 1000 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [1000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_1000'},
    
    'GEMB_FR_3_5_1100': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 1100 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [1100],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_1100'},
    
     'GEMB_FR_3_5_1200': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 1200 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [1200],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_1200'},
    
    'GEMB_FR_3_5_1300': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 1300 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [1300],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_1300'},
    
      'GEMB_FR_3_5_1400': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 1400 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [1400],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_1400'},
    
 'GEMB_FR_3_5_1500': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 1500 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [1500],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_1500'},
    
    'GEMB_FR_3_5_1600': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 1600 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [1600],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_1600'},
     
    'GEMB_FR_3_5_1700': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 1700 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [1700],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_1700'},
    
     'GEMB_FR_3_5_1800': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 1800 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [1800],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_1800'},
    
    'GEMB_FR_3_5_1900': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 1900 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [1900],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_1900'},
    
 'GEMB_FR_3_5_2000': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 2000 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [2000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_2000'},
    
    'GEMB_FR_3_5_2100': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 2100 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [2100],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_2100'},
    
    'GEMB_FR_3_5_2200': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 2200 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [2200],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_2200'},
    
    'GEMB_FR_3_5_2300': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 2300 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [2300],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_2300'},
    
    'GEMB_FR_3_5_2400': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 2400 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [2400],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_2400'},
    
      'GEMB_FR_3_5_2500': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 2500 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [2500],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_2500'},
    
    'GEMB_FR_3_5_2600': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 2600 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [2600],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_2600'},
    
    'GEMB_FR_3_5_2700': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 2700 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [2700],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_2700'},
    
    'GEMB_FR_3_5_2800': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 2800 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [2800],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_2800'},
    
    'GEMB_FR_3_5_2900': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 2900 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [2900],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_2900'},
    
 'GEMB_FR_3_5_3000': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 3000 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [3000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_3000'},
 'GEMB_FR_3_5_3100': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 3100 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [3100],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_3100'},
     'GEMB_FR_3_5_3200': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 3200 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [3200],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_3200'},
    'GEMB_FR_3_5_3300': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 3300 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [3300],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_3300'},
      'GEMB_FR_3_5_3400': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 3300 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [3400],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_3400'},
        'GEMB_FR_3_5_3500': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 3500 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [3500],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_3500'},
    'GEMB_FR_3_5_3600': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 3600 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [3600],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_3600'},
       'GEMB_FR_3_5_3700': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 3700 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [3700],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_3700'},
        'GEMB_FR_3_5_3800': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 3800 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [3800],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_3800'},
        'GEMB_FR_3_5_3900': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 3900 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [3900],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_3900'},
     'GEMB_FR_3_5_4000': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 4000 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [4000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_4000'},
    
 'GEMB_FR_3_5_5000': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 5000 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [5000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_5000'},
 'GEMB_FR_3_5_8000': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] 8000 samplesand transfer learning from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': [8000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_8000'},
 'GEMB_FR_3_5_ALL': {'run': True,
  'name' : 'GEMB_FR_3_5',                 
  'description': "GoEmo with embeddings frozen followed by Friends with layers 3 to 5 frozen with ['tales'] ALL samples and xfer learn from goemb_frall_tr3_5",
  'load_model': 'goemb_frall_tr3_5',
  'train_corpus': ['tales'],
  'train_samples': None,
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_fr_3_5_all'},
 
    'GO_SEM_FR_100': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 100 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [100],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_100'},
    
      'GO_SEM_FR_200': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 200 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [200],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_200'},
    'GO_SEM_FR_300': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 300 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [300],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_300'},
    'GO_SEM_FR_400': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 400 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [400],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_400'},
        
    'GO_SEM_FR_500': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 500 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [500],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_500'},
    
    'GO_SEM_FR_600': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 600 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [600],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_600'},
    'GO_SEM_FR_700': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 700 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [700],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_700'},
    'GO_SEM_FR_800': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 800 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [800],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_800'},
    'GO_SEM_FR_900': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 900 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [900],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_900'},
    
 'GO_SEM_FR_1000': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 1000 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [1000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_1000'},
    
  'GO_SEM_FR_1100': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 1100 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [1100],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_1100'}, 
    'GO_SEM_FR_1200': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 1200 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [1200],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_1200'}, 
    'GO_SEM_FR_1300': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 1300 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [1300],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_1300'}, 
     'GO_SEM_FR_1400': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 1400 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [1400],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_1400'}, 
 'GO_SEM_FR_1500': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 1500 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [1500],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_1500'},
    
'GO_SEM_FR_1600': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 1600 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [1600],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_1600'},
    
'GO_SEM_FR_1700': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 1600 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [1700],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_1700'},
    
    'GO_SEM_FR_1800': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 1800 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [1800],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_1800'},
    'GO_SEM_FR_1900': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 1900 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [1900],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_1900'},
    'GO_SEM_FR_2000': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 2000 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [2000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_2000'},
    
    'GO_SEM_FR_2100': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 2100 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [2100],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_2100'},
    
      'GO_SEM_FR_2200': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 2200 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [2200],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_2200'},
      'GO_SEM_FR_2300': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 2300 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [2300],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_2300'},
    'GO_SEM_FR_2400': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 2500 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [2400],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_2400'},
    
    'GO_SEM_FR_2500': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 2500 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [2500],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_2500'},
    'GO_SEM_FR_2600': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 2600 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [2600],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_2600'},
    
    'GO_SEM_FR_2700': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 2700 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [2700],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_2700'},
    
    'GO_SEM_FR_2800': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 2800 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [2800],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_2800'},
    
    'GO_SEM_FR_2900': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 2900 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [2900],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_2900'},
    
 'GO_SEM_FR_3000': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 3000 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [3000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_3000'},
    
    'GO_SEM_FR_3100': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 3100 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [3100],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_3100'},
     'GO_SEM_FR_3200': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 3200 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [3200],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_3200'},
    'GO_SEM_FR_3300': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 3300 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [3300],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_3300'},
      'GO_SEM_FR_3400': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 3400 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [3400],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_3400'},
       'GO_SEM_FR_3500': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 3500 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [3500],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_3500'},
       'GO_SEM_FR_3600': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 3600 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [3600],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_3600'},
      'GO_SEM_FR_3700': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 3700 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [3700],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_3700'},
       'GO_SEM_FR_3800': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 3800 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [3800],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_3800'},
      'GO_SEM_FR_3900': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 3900 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [3900],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_3800'},
     'GO_SEM_FR_4000': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 4000 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [4000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_4000'},
    
    
 'GO_SEM_FR_5000': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 5000 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [5000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_5000'},
 'GO_SEM_FR_8000': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] 8000 samplesand transfer learning from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': [8000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_8000'},
 'GO_SEM_FR_ALL': {'run': True,
  'name' : 'GO_SEM_FR',                 
  'description': "GoEmo, SemEval and Friends  with ['tales'] ALL samples and xfer learn from emo_goemo_sem_fr",
  'load_model': 'emo_goemo_sem_fr',
  'train_corpus': ['tales'],
  'train_samples': None,
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_fr_all'},
 
    'GO_FR_100': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 100 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [100],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_100'},
     'GO_FR_200': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 200 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [200],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_200'},
         'GO_FR_300': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 300 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [300],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_300'},
    'GO_FR_400': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 400 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [400],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_400'},
    
    'GO_FR_500': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 500 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [500],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_500'},
    
    'GO_FR_600': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 600 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [600],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_600'},
    'GO_FR_700': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 700 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [700],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_700'},
     'GO_FR_800': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 800 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [800],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_800'},
     'GO_FR_900': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 900 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [900],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_900'},
        
 'GO_FR_1000': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 1000 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [1000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_1000'},
    
    'GO_FR_1100': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 1100 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [1100],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_1100'},
    
       'GO_FR_1200': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 1200 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [1200],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_1200'},
    
      'GO_FR_1300': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 1300 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [1300],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_1300'},
    
    'GO_FR_1400': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 1400 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [1400],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_1400'},
    
     'GO_FR_1500': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 1500 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [1500],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_1500'},
    
 'GO_FR_1600': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 1600 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [1600],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_1600'},
    'GO_FR_1700': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 1700 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [1700],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_1700'},
    
       'GO_FR_1800': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 1800 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [1800],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_1800'},
    
 'GO_FR_1900': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 1800 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [1900],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_1900'},
 'GO_FR_2000': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 2000 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [2000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_2000'},
    
    'GO_FR_2100': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 2100 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [2100],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_2100'},
    
     'GO_FR_2200': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 2200 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [2200],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_2200'},
     'GO_FR_2300': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 2300 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [2300],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_2300'},
     'GO_FR_2400': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 2400 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [2400],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_2400'},
      'GO_FR_2500': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 2500 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [2500],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_2500'},
    
    'GO_FR_2600': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 2500 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [2600],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_2600'},
     'GO_FR_2700': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 2700 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [2700],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_2700'},
     'GO_FR_2800': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 2800 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [2800],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_2800'},
      'GO_FR_2900': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 2900 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [2900],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_2900'},
    
    'GO_FR_3000': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 3000 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [3000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_3000'},
     'GO_FR_3100': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 3100 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [3100],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_3100'}, 
       'GO_FR_3200': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 3200 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [3200],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_3200'}, 
    
    'GO_FR_3300': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 3300 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [3300],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_3300'}, 
    'GO_FR_3400': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 3400 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [3400],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_3400'}, 
     'GO_FR_3500': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 3500 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [3500],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_3500'},
    'GO_FR_3600': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 3600 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [3600],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_3600'},
    'GO_FR_3700': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 3700 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [3700],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_3700'},
    
    'GO_FR_3800': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 3700 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [3800],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_3800'},
    'GO_FR_3900': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 3900 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [3900],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_3900'},
    'GO_FR_4000': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 4000 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [4000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_4000'},
    
 'GO_FR_5000': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 5000 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [5000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_5000'},
 'GO_FR_8000': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] 8000 samplesand transfer learning from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': [8000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_8000'},
 'GO_FR_ALL': {'run': True,
  'name' : 'GO_FR',                 
  'description': "GoEmo, Friends  with ['tales'] ALL samples and xfer learn from emo_goemo_fr",
  'load_model': 'emo_goemo_fr',
  'train_corpus': ['tales'],
  'train_samples': None,
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_fr_all'},

    ######COPY 2 will start from HERE########
    'GO_SEM_500': {'run': False,
  'name' : 'GO_SEM',                 
  'description': "GoEmo, SemEval  with ['tales'] 500 samplesand transfer learning from emo_goemo_sem",
  'load_model': 'emo_goemo_sem',
  'train_corpus': ['tales'],
  'train_samples': [500],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_500'},
 'GO_SEM_1000': {'run': True,
  'name' : 'GO_SEM',                 
  'description': "GoEmo, SemEval  with ['tales'] 1000 samplesand transfer learning from emo_goemo_sem",
  'load_model': 'emo_goemo_sem',
  'train_corpus': ['tales'],
  'train_samples': [1000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_1000'},
 'GO_SEM_1500': {'run': False,
  'name' : 'GO_SEM',                 
  'description': "GoEmo, SemEval  with ['tales'] 1500 samplesand transfer learning from emo_goemo_sem",
  'load_model': 'emo_goemo_sem',
  'train_corpus': ['tales'],
  'train_samples': [1500],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_1500'},
 'GO_SEM_2000': {'run': False,
  'name' : 'GO_SEM',                 
  'description': "GoEmo, SemEval  with ['tales'] 2000 samplesand transfer learning from emo_goemo_sem",
  'load_model': 'emo_goemo_sem',
  'train_corpus': ['tales'],
  'train_samples': [2000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_2000'},
 'GO_SEM_3000': {'run': False,
  'name' : 'GO_SEM',                 
  'description': "GoEmo, SemEval  with ['tales'] 3000 samplesand transfer learning from emo_goemo_sem",
  'load_model': 'emo_goemo_sem',
  'train_corpus': ['tales'],
  'train_samples': [3000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_3000'},
 'GO_SEM_5000': {'run': False,
  'name' : 'GO_SEM',                 
  'description': "GoEmo, SemEval  with ['tales'] 5000 samplesand transfer learning from emo_goemo_sem",
  'load_model': 'emo_goemo_sem',
  'train_corpus': ['tales'],
  'train_samples': [5000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_5000'},
 'GO_SEM_8000': {'run': False,
  'name' : 'GO_SEM',                 
  'description': "GoEmo, SemEval  with ['tales'] 8000 samplesand transfer learning from emo_goemo_sem",
  'load_model': 'emo_goemo_sem',
  'train_corpus': ['tales'],
  'train_samples': [8000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_8000'},
 'GO_SEM_ALL': {'run': False,
  'name' : 'GO_SEM',                 
  'description': "GoEmo, SemEval  with ['tales'] ALL samples and xfer learn from emo_goemo_sem",
  'load_model': 'emo_goemo_sem',
  'train_corpus': ['tales'],
  'train_samples': False,
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_sem_all'},
 'GO_500': {'run': False,
  'name' : 'GO',                 
  'description': "GoEmo  with ['tales'] 500 samplesand transfer learning from emo_goemo",
  'load_model': 'emo_goemo',
  'train_corpus': ['tales'],
  'train_samples': [500],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_500'},
 'GO_1000': {'run': False,
  'name' : 'GO',                 
  'description': "GoEmo  with ['tales'] 1000 samplesand transfer learning from emo_goemo",
  'load_model': 'emo_goemo',
  'train_corpus': ['tales'],
  'train_samples': [1000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_1000'},
 'GO_1500': {'run': False,
  'name' : 'GO',                 
  'description': "GoEmo  with ['tales'] 1500 samplesand transfer learning from emo_goemo",
  'load_model': 'emo_goemo',
  'train_corpus': ['tales'],
  'train_samples': [1500],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_1500'},
 'GO_2000': {'run': False,
  'name' : 'GO',                 
  'description': "GoEmo  with ['tales'] 2000 samplesand transfer learning from emo_goemo",
  'load_model': 'emo_goemo',
  'train_corpus': ['tales'],
  'train_samples': [2000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_2000'},
 'GO_3000': {'run': False,
  'name' : 'GO',                 
  'description': "GoEmo  with ['tales'] 3000 samplesand transfer learning from emo_goemo",
  'load_model': 'emo_goemo',
  'train_corpus': ['tales'],
  'train_samples': [3000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_3000'},
 'GO_5000': {'run': False,
  'name' : 'GO',                 
  'description': "GoEmo  with ['tales'] 5000 samplesand transfer learning from emo_goemo",
  'load_model': 'emo_goemo',
  'train_corpus': ['tales'],
  'train_samples': [5000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_5000'},
 'GO_8000': {'run': False,
  'name' : 'GO',                 
  'description': "GoEmo  with ['tales'] 8000 samplesand transfer learning from emo_goemo",
  'load_model': 'emo_goemo',
  'train_corpus': ['tales'],
  'train_samples': [8000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_8000'},
 'GO_ALL': {'run': False,
  'name' : 'GO',                 
  'description': "GoEmo  with ['tales'] ALL samples and xfer learn from emo_goemo",
  'load_model': 'emo_goemo',
  'train_corpus': ['tales'],
  'train_samples': None,
  'test_corpus': ['tales'],
  'save_stats': 'stats_go_all'},
'GEMB_500': {'run': True,
  'name': 'GEMB',
  'description': "GoEmo with embeddings frozen with ['tales'] 500 samplesand transfer learning from emo_goemo_emb_fr",
  'load_model': 'emo_goemo_emb_fr',
  'train_corpus': ['tales'],
  'train_samples': [500],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_500'},
 'GEMB_1000': {'run': False,
  'name': 'GEMB',
  'description': "GoEmo with embeddings frozen with ['tales'] 1000 samplesand transfer learning from emo_goemo_emb_fr",
  'load_model': 'emo_goemo_emb_fr',
  'train_corpus': ['tales'],
  'train_samples': [1000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_1000'},
 'GEMB_1500': {'run': False,
  'name': 'GEMB',
  'description': "GoEmo with embeddings frozen with ['tales'] 1500 samplesand transfer learning from emo_goemo_emb_fr",
  'load_model': 'emo_goemo_emb_fr',
  'train_corpus': ['tales'],
  'train_samples': [1500],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_1500'},
 'GEMB_2000': {'run': False,
  'name': 'GEMB',
  'description': "GoEmo with embeddings frozen with ['tales'] 2000 samplesand transfer learning from emo_goemo_emb_fr",
  'load_model': 'emo_goemo_emb_fr',
  'train_corpus': ['tales'],
  'train_samples': [2000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_2000'},
 'GEMB_3000': {'run': False,
  'name': 'GEMB',
  'description': "GoEmo with embeddings frozen with ['tales'] 3000 samplesand transfer learning from emo_goemo_emb_fr",
  'load_model': 'emo_goemo_emb_fr',
  'train_corpus': ['tales'],
  'train_samples': [3000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_3000'},
 'GEMB_5000': {'run': False,
  'name': 'GEMB',
  'description': "GoEmo with embeddings frozen with ['tales'] 5000 samplesand transfer learning from emo_goemo_emb_fr",
  'load_model': 'emo_goemo_emb_fr',
  'train_corpus': ['tales'],
  'train_samples': [5000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_5000'},
 'GEMB_8000': {'run': False,
  'name': 'GEMB',
  'description': "GoEmo with embeddings frozen with ['tales'] 8000 samplesand transfer learning from emo_goemo_emb_fr",
  'load_model': 'emo_goemo_emb_fr',
  'train_corpus': ['tales'],
  'train_samples': [8000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_8000'},
 'GEMB_ALL': {'run': False,
  'name': 'GEMB',
  'description': "GoEmo with embeddings frozen with ['tales'] ALL samples and xfer learn from emo_goemo_emb_fr",
  'load_model': 'emo_goemo_emb_fr',
  'train_corpus': ['tales'],
  'train_samples': False,
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_all'},
 'GEMB_SEMFR_0_5_500': {'run': False,
  'name': 'GEMB_SEMFR_0_5',
  'description': "GoEmo, SemEval, Friends with embeddings and all layers frozen with ['tales'] 500 samplesand transfer learning from goemb_semfrall_tr0_5",
  'load_model': 'goemb_semfrall_tr0_5',
  'train_corpus': ['tales'],
  'train_samples': [500],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_semfr_0_5_500'},
 'GEMB_SEMFR_0_5_1000': {'run': False,
  'name': 'GEMB_SEMFR_0_5',
  'description': "GoEmo, SemEval, Friends with embeddings and all layers frozen with ['tales'] 1000 samplesand transfer learning from goemb_semfrall_tr0_5",
  'load_model': 'goemb_semfrall_tr0_5',
  'train_corpus': ['tales'],
  'train_samples': [1000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_semfr_0_5_1000'},
 'GEMB_SEMFR_0_5_1500': {'run': False,
  'name': 'GEMB_SEMFR_0_5',
  'description': "GoEmo, SemEval, Friends with embeddings and all layers frozen with ['tales'] 1500 samplesand transfer learning from goemb_semfrall_tr0_5",
  'load_model': 'goemb_semfrall_tr0_5',
  'train_corpus': ['tales'],
  'train_samples': [1500],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_semfr_0_5_1500'},
 'GEMB_SEMFR_0_5_2000': {'run': False,
  'name': 'GEMB_SEMFR_0_5',
  'description': "GoEmo, SemEval, Friends with embeddings and all layers frozen with ['tales'] 2000 samplesand transfer learning from goemb_semfrall_tr0_5",
  'load_model': 'goemb_semfrall_tr0_5',
  'train_corpus': ['tales'],
  'train_samples': [2000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_semfr_0_5_2000'},
 'GEMB_SEMFR_0_5_3000': {'run': False,
  'name': 'GEMB_SEMFR_0_5',
  'description': "GoEmo, SemEval, Friends with embeddings and all layers frozen with ['tales'] 3000 samplesand transfer learning from goemb_semfrall_tr0_5",
  'load_model': 'goemb_semfrall_tr0_5',
  'train_corpus': ['tales'],
  'train_samples': [3000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_semfr_0_5_3000'},
 'GEMB_SEMFR_0_5_5000': {'run': False,
  'name': 'GEMB_SEMFR_0_5',
  'description': "GoEmo, SemEval, Friends with embeddings and all layers frozen with ['tales'] 5000 samplesand transfer learning from goemb_semfrall_tr0_5",
  'load_model': 'goemb_semfrall_tr0_5',
  'train_corpus': ['tales'],
  'train_samples': [5000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_semfr_0_5_5000'},
 'GEMB_SEMFR_0_5_8000': {'run': False,
  'name': 'GEMB_SEMFR_0_5',
  'description': "GoEmo, SemEval, Friends with embeddings and all layers frozen with ['tales'] 8000 samplesand transfer learning from goemb_semfrall_tr0_5",
  'load_model': 'goemb_semfrall_tr0_5',
  'train_corpus': ['tales'],
  'train_samples': [8000],
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_semfr_0_5_8000'},
 'GEMB_SEMFR_0_5_ALL': {'run': False,
  'name': 'GEMB_SEMFR_0_5',
  'description': "GoEmo, SemEval, Friends with embeddings and all layers frozen with ['tales'] ALL samples and xfer learn from goemb_semfrall_tr0_5",
  'load_model': 'goemb_semfrall_tr0_5',
  'train_corpus': ['tales'],
  'train_samples': None,
  'test_corpus': ['tales'],
  'save_stats': 'stats_gemb_semfr_0_5_all'}    
}        

run_exp = {k:v for k,v in experiments.items() if v['run']}
for i,k in enumerate(run_exp.keys()):
    run_exp[k]['run_step'] = i
#We don't expect actual job to be run within an hour. 
# So re-write config, if being run within hour
with open('config_{}0000.json'.format(time.strftime('%Y%m%d_%H')),'w') as f:
    config = {**metadata,**model_config,**run_exp}
    json.dump(config,f)
print (len(experiments.keys()))

209


In [29]:
def validate_config():
    def getIssue(issue_type,config,key,problem,desc):
        issue = {}
        issue['type'] = issue_type
        issue['config']=config
        issue['key'] =key
        issue['issue'] =problem
        issue['desc'] =desc
        return issue

    # Metadata runtype should not have spaces
    issues = []
    if re.findall('\s+',metadata['run_type']):
        iss = getIssue('WARN','metadata','run_type',
                'White space found','this may create issues in file creation')
        issues.append(iss)
    if model_config['epochs'] < 4:
        iss = getIssue('WARN','model_config','epochs',
                'Very few epochs','Epochs specified is < normal 4. Use Trial Run feature to test the run.')
        issues.append(iss)

    exp_keys = list(run_exp.keys())
    required_keys = ['dataset_path',
                'input_model_path',
                'model_path',
                'output_path',
                'test_corpus']
    train_req_keys = ['epochs',
                    'batch_size',
                    'max_length',
                    'lr',
                    'drop_out']

    for i,e in enumerate(exp_keys):
        ec = {}

        ec.update(model_config)
        ec.update(run_exp[e])

        #All required keys MUST be present in config
        missing = [key for key in required_keys if key not in ec]
        if missing:
            iss = getIssue('ERROR', e, ','.join(missing),
                'Required keys missing','Provide missing keys')
            issues.append(iss)

        load_model = ec.get('load_model',None)
        if load_model:
            #If load model is specified, check if a prior step is creating the model
            found = False 
            for j in range(i):
                sm = run_exp[exp_keys[j]].get('save_model',None)
                if sm == load_model:
                    found = True 
            if not found:
                load_model_f = ec['input_model_path'] + '/' + load_model + metadata['model_ext']
                found = os.path.exists(load_model_f)
            if not found:
                load_model_f = ec['dataset_path'] + '/' + load_model + metadata['model_ext']
                found = os.path.exists(load_model_f)
            if not found:
                iss = getIssue('ERROR',e,'load_model',
                      'Load Model not found',
                      '{} not found in previous experiments or as file {}'.format(
                          load_model,load_model_f))
                issues.append(iss)
        if ec.get('train_corpus',None):
            missing = [key for key in train_req_keys if key not in ec]
            if missing:
                iss = getIssue('ERROR', e, ','.join(missing),
                'Required keys missing','These keys required when training data is specified')
                issues.append(iss)
            tc = ec['train_corpus']
            ts = ec.get('train_samples',None)
            if not isinstance(tc,list):
                iss = getIssue('ERROR',e,'train_corpus',
                        'Training corpus must be a list',
                        'Training corpus {} is not a list'.format(tc)) 
                issues.append(iss)

            if ts:
                if not isinstance(ts,list):
                    iss = getIssue('ERROR',e,'train_samples',
                            'Training samples must be a list',
                            'Training samples {} is not a list'.format(ts)) 
                    issues.append(iss)
                if isinstance(tc,list) and isinstance(ts,list):
                    if len(tc) != len(ec['train_samples']):
                        iss = getIssue('ERROR',e,'train_corpus,train_samples',
                                'Training corpus, samples length mismatch',
                                'Training corpus {}, samples {}'.format(
                                    ','.join(tc),','.join(ec['train_samples']))) 
                        issues.append(iss)
            for fc in tc:
                fl = [ec['dataset_path'] +'/' + fc + '_train.tsv',ec['dataset_path'] +'/' + fc + '_dev.tsv']
                for f in fl:
                    if not os.path.exists(f):
                        iss = getIssue('ERROR',e,'train_corpus',
                                'Train file not found',
                                'Training corpus {}. File {} not found'.format(
                                    ','.join(tc),f))
                        issues.append(iss)

        if ec.get('test_corpus',None):
            tc = ec['test_corpus']
            ts = ec.get('test_samples',None)
            if not isinstance(tc,list):
                iss = getIssue('ERROR',e,'test_corpus',
                        'Test corpus must be a list',
                        'Test corpus {} is not a list'.format(tc)) 
                issues.append(iss)
            if ts:
                if not isinstance(ts,list):
                    iss = getIssue('ERROR',e,'train_samples',
                            'Training samples must be a list',
                            'Training samples {} is not a list'.format(ts)) 
                    issues.append(iss)
                if isinstance(tc,list) and isinstance(ts,list):
                    if len(tc) != len(ec['test_samples']):
                        iss = getIssue('ERROR',e,'test_corpus,test_samples',
                                'Test corpus, samples length mismatch',
                                'Test corpus {}, samples {}'.format(
                                    ','.join(tc),','.join(ec['test_samples'])))
                        issues.append(iss)
            if i > 0:
                ps = run_exp[exp_keys[i-1]].get('test_samples',None)
                if ts != ps :
                    iss = getIssue('WARN',e,'test_samples',
                        'Test sample does not match previous',
                        'Test sample {} for {} does not match {} s sample {}'.format(e,ts,exp_keys[i-1],ps))
                    issues.append(iss)
            for fc in tc:
                f = ec['dataset_path'] +'/' + fc + '_test.tsv'
                if not os.path.exists(f):
                    iss = getIssue('ERROR',e,'test_corpus',
                          'Test file not found',
                          'Test corpus {}. File {} not found'.format(','.join(tc),f))
                    issues.append(iss)
        else:
            iss = getIssue('ERROR',e,'test_corpus',
                'Test corpus is empty',
                'Test corpus cannot be None. Must be specified')
            issues.append(iss) 
    return issues                                       

issues = validate_config()
errors = [iss for iss in issues if iss['type'] == 'ERROR']
warnings = [iss for iss in issues if iss['type'] == 'WARN']
print ('{} Errors and {} Warning found'.format(len(errors),len(warnings)))
print ('ERRORS:')
pprint.pprint(errors)
print ('')
print ('WARNINGS:')
pprint.pprint(warnings)

0 Errors and 0 Warning found
ERRORS:
[]

WARNINGS:
[]


In [30]:
set([k['desc'] for k in errors])

set()

In [31]:
if metadata['TRIAL_RUN']:
    trial_size = metadata['TRIAL_SIZE']
    print ('Trial run {} experiments with Trial Size {}'.format(len(run_exp.keys()),trial_size))
    print ('Changing epochs from {} to {} '.format(model_config['epochs'],metadata['TRIAL_EPOCHS']))
    model_config['epochs'] = metadata['TRIAL_EPOCHS']
    for e,c in run_exp.items():
        if c['train_corpus']:
            c['train_samples'] = [trial_size] * len(c['train_corpus'])
        if c['test_corpus']:
            c['test_samples'] = [trial_size] * len(c['test_corpus'])
else:
    print ('Production {} experiments'.format(len(run_exp.keys())))
print ("\n**** Model_Config ****")
pprint.pprint(model_config)
print ("\n**** Run Experiments ****")
pprint.pprint(list(run_exp.keys()))
pprint.pprint(run_exp)

Production 174 experiments

**** Model_Config ****
{'batch_size': 16,
 'dataset_path': 'Data/',
 'drop_out': 0.2,
 'epochs': 4,
 'input_model_path': './models',
 'lr': 4e-05,
 'max_length': 512,
 'model_path': './models',
 'output_path': './training_results/clean_runs'}

**** Run Experiments ****
['TALES_100',
 'TALES_200',
 'TALES_300',
 'TALES_400',
 'TARGET_500',
 'TALES_600',
 'TALES_700',
 'TALES_800',
 'TALES_900',
 'TALES_1000',
 'TALES_1100',
 'TALES_1200',
 'TALES_1300',
 'TALES_1400',
 'TALES_1500',
 'TALES_1600',
 'TALES_1700',
 'TALES_1800',
 'TALES_1900',
 'TALES_2000',
 'TALES_2100',
 'TALES_2200',
 'TALES_2300',
 'TALES_2400',
 'TALES_2500',
 'TALES_2600',
 'TALES_2700',
 'TALES_2800',
 'TALES_2900',
 'TALES_3000',
 'TALES_3100',
 'TALES_3200',
 'TALES_3300',
 'TALES_3400',
 'TALES_3500',
 'TALES_3600',
 'TALES_3700',
 'TALES_3800',
 'TALES_3900',
 'TALES_4000',
 'TALES_5000',
 'TALES_8000',
 'TALES_ALL',
 'GEMB_FR_3_5_100',
 'GEMB_FR_3_5_200',
 'GEMB_FR_3_5_300',
 'GEMB

In [32]:
print ('TRIAL MODE: {} DEVICE: {} Experiments:{} '.format(
       metadata['TRIAL_RUN'],device,len(run_exp.keys())))


TRIAL MODE: False DEVICE: cuda Experiments:174 


In [33]:
def genexp(name,desc,load_model,sizes=[0,100,200,300],train=['tales'],test=['tales'],gen_all=True):
    exp = {}
    for size in sizes:
        expname = name.upper() + '_' + str(size)
        exp[expname] = {}
        exp[expname]['run'] = True
        exp[expname]['name'] = name
        exp[expname]['description'] = desc + ' with ' + str(train) + ' ' + str(size) + ' samples' 
        if load_model:
            exp[expname]['description'] += 'and transfer learning from ' + load_model 
        exp[expname]['load_model'] = load_model
        
        if size == 0 :
            exp[expname]['train_corpus'] = None
            exp[expname]['train_samples'] = None
        else:
            exp[expname]['train_corpus'] = train
            exp[expname]['train_samples'] = [size] * len(train)
        exp[expname]['test_corpus'] = test
        exp[expname]['save_stats'] = 'stats_' + expname.lower()
    if gen_all:
        expname = name.upper() + '_ALL'
        exp[expname] = {}
        exp[expname]['run'] = True
        exp[expname]['name'] = name
        exp[expname]['description'] = desc + ' with ' + str(train) + ' ALL samples'
        if load_model:
            exp[expname]['description'] += ' and xfer learn from ' + load_model 
        exp[expname]['load_model'] = load_model
        exp[expname]['train_corpus'] = train
        exp[expname]['train_samples'] = None
        exp[expname]['test_corpus'] = test
        exp[expname]['save_stats'] = 'stats_' + expname.lower()
    return exp

all_exp = {}
sizes = [0,100,200,300,400,500,1000,1500,2000,2500,3000,3500,4000,4500,5000,6000,7000,8000]
all_exp.update(genexp('TALES','Tales Only',None,gen_all=False,sizes=sizes[1:]))
#These produced positive results
# all_exp.update(genexp('GEMB_FR_3_5','GoEmo, Friends with embeddings and layers 3 to 5 frozen','goemb_frall_tr3_5',sizes=sizes))
# all_exp.update(genexp('GO_SEM_FR','GoEmo, SemEval and Friends ','emo_goemo_sem_fr',sizes=sizes))
# all_exp.update(genexp('GO_FR','GoEmo, Friends ','emo_goemo_fr',sizes=sizes))
# all_exp.update(genexp('GO_SEM','GoEmo, SemEval ','emo_goemo_sem',sizes=sizes))
# all_exp.update(genexp('GO','GoEmo ','emo_goemo',sizes=sizes))
# #These produced negative results
all_exp.update(genexp('GEMB','GoEmo with embeddings frozen','emo_goemo_emb_fr',sizes=sizes))
all_exp.update(genexp('GEMB_SEM_FR_0_5','GoEmo, SemEval, Friends with embeddings and all layers frozen','goemb_semfrall_tr0_5',sizes=sizes))

#all_exp 

In [ ]:
run_stats = []

if len(errors) == 0:
    for e,c in all_exp.items():
        ec = {}
        ec.update({'experiment':e})        
        ec.update(model_config)
        ec.update(c)
    
        print ('Running Experiment',e)
        model = None
        train_stats = []
        if ec.get('load_model',None):
            model_name = ec['load_model']+metadata['model_ext']
            out_model = ec['model_path'] + '/' + model_name
            in_model = ec['input_model_path'] + '/' + model_name
            if os.path.exists(out_model):
                model = torch.load(out_model,map_location=device)
            elif os.path.exists(in_model):
                model = torch.load(in_model,map_location=device)
            else:
                model = torch.load(ec['dataset_path'] + '/' + model_name,map_location=device)
        trained_model = model 
        if ec.get('train_corpus',None):
            train_data = load_corpusdata(ec['dataset_path'],
                                            corpus=ec['train_corpus'],
                                            samples=ec.get('train_samples',None),
                                            type='train')
            dev_data = load_corpusdata(ec['dataset_path'],
                                        corpus=ec['train_corpus'],
                                        samples=ec.get('train_samples',None),
                                        type='dev')
            trained_model, train_stats = train_model(train_data,dev_data,model,
                                    freeze_layers=ec.get('freeze_layers',None),
                                        epochs=ec['epochs'],
                                        batch_size=ec['batch_size'],
                                        drop_out=ec['drop_out'],
                                        lr=ec['lr'],
                                        max_length=ec['max_length'])
            train_predict_results = test_model(train_data,trained_model,epoch='tr_predict',
                                        max_length=ec['max_length'])
            train_stats.append(train_predict_results)

        test_data = load_corpusdata(ec['dataset_path'],
                                    corpus=ec['test_corpus'],
                                    samples=ec.get('test_samples',None),
                                    type='test')
        test_results = test_model(test_data,trained_model,epoch='test',
                                    max_length=ec['max_length'])
        train_stats.append(test_results)
        for d in train_stats:
            d.update(ec)
        if ec.get('save_model',None):
            #https://stackoverflow.com/questions/42703500/best-way-to-save-a-trained-model-in-pytorch
            torch.save(trained_model.state_dict(),
                ec['model_path'] + '/' + ec['save_model']+metadata['dict_ext'])
            torch.save(trained_model,
                ec['model_path'] + '/' + ec['save_model']+metadata['model_ext'])
        if ec.get('save_stats',None):
            pdt = pd.DataFrame(train_stats)
            pdt.to_csv(ec['output_path'] + '/' + ec['save_stats'] + '.csv')

        run_stats.extend(train_stats)  

    pd_results = pd.DataFrame(run_stats)
    pd_results.to_csv(
        os.path.join(OUTPUT_PATH,
                    'results_{}_{}.csv'.format(metadata['run_type'],time.strftime('%Y%m%d_%H%M%S'))))        

Running Experiment TALES_100
*****  Epoch 0 *****
-----  Training -----
Avg Training Loss 0.5981, Completed in 0:00:01.897979 
-----  Validation -----
**** Loss ******
Avg Validation Loss 0.4478, Completed in 0:00:00.564987 
**** Results ******
F1 Score (weighted) =   0.319444
F1 Score (micro) =   0.474227
F1 Score (macro) =   0.091270
F1 Score (samples) =   0.460000
*****  Epoch 1 *****
-----  Training -----
Avg Training Loss 0.3727, Completed in 0:00:01.686528 
-----  Validation -----
**** Loss ******
Avg Validation Loss 0.3399, Completed in 0:00:00.565560 
**** Results ******
F1 Score (weighted) =   0.333333
F1 Score (micro) =   0.500000
F1 Score (macro) =   0.095238
F1 Score (samples) =   0.500000
*****  Epoch 2 *****
-----  Training -----
Avg Training Loss 0.3104, Completed in 0:00:01.687071 
-----  Validation -----
**** Loss ******
Avg Validation Loss 0.3550, Completed in 0:00:00.565189 
**** Results ******
F1 Score (weighted) =   0.333333
F1 Score (micro) =   0.500000
F1 Score (

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


Avg Training Loss 0.1536, Completed in 0:00:01.558450 
-----  Validation -----
**** Loss ******
Avg Validation Loss 0.4212, Completed in 0:00:00.582281 
**** Results ******
F1 Score (weighted) =   0.350950
F1 Score (micro) =   0.433862
F1 Score (macro) =   0.201289
F1 Score (samples) =   0.410000
*****  Epoch 2 *****
-----  Training -----
Avg Training Loss 0.1113, Completed in 0:00:01.557919 
-----  Validation -----
**** Loss ******
Avg Validation Loss 0.4254, Completed in 0:00:00.580449 
**** Results ******
F1 Score (weighted) =   0.345960
F1 Score (micro) =   0.425532
F1 Score (macro) =   0.199550
F1 Score (samples) =   0.400000
*****  Epoch 3 *****
-----  Training -----
Avg Training Loss 0.0780, Completed in 0:00:01.556777 
-----  Validation -----
**** Loss ******
Avg Validation Loss 0.4544, Completed in 0:00:00.585771 
**** Results ******
F1 Score (weighted) =   0.375139
F1 Score (micro) =   0.442105
F1 Score (macro) =   0.221402
F1 Score (samples) =   0.420000
-----  Testing -----

In [ ]:
model = get_basemodel(7,0.2)

In [ ]:
for name,param in model.named_parameters():
    print (name,param.size(),'transformer.layer.0' in name)

In [ ]:
freeze_layers = ['embeddings','transformer']
for name,param in model.named_parameters():
    if any([fl in name for fl in freeze_layers]):
        param.requires_grad = False
    